In [1]:
from sympy import *
from IPython.display import display

def quality_evaluation(starting_tier, target_tier, max_qual, base_prod, prods, quals, debug = False, gradients = False):
    h = 5
    m = target_tier
    #h, m = symbols("h, m", integer=True, positive=True, nonzero=True)
    T4qual = Rational(1, 40) * (Rational(5, 2) if max_qual else Rational(1, 1)) # type: ignore
    T4prod = Rational(1, 10) * (Rational(5, 2) if max_qual else Rational(1, 1)) # type: ignore
    q_cv, p_cv, q_rv, p_rv = quals * T4qual, (1.5 if base_prod else 1.0) + prods * T4prod, 4 * T4qual, 1 # type: ignore
    d_0v = [0, 0, 0, 0, 0]
    d_0v[starting_tier] = 1

    q_c, p_c, q_r, p_r = symbols("q_c, p_c, q_r, p_r", real=True, positive=True)
    q_cs = Matrix([[*symbols(", ".join("q_{c_"+str(i)+"}" for i in range(1, h+1)), real=True, positive=True)]])
    p_cs = Matrix([[*symbols(", ".join("p_{c_"+str(i)+"}" for i in range(1, h+1)), real=True, positive=True)]])
    q_rs = Matrix([[*symbols(", ".join("q_{r_"+str(i)+"}" for i in range(1, h+1)), real=True, positive=True)]])
    p_rs = Matrix([[*symbols(", ".join("p_{r_"+str(i)+"}" for i in range(1, h+1)), real=True, positive=True)]])

    #display(q_cs)

    d_0 = Matrix([[*symbols(", ".join("d_0"+str(i) for i in range(1, h+1)), integer=True, positive=True, nonzero=True)]]).T

    l = Matrix([[1]*(m-1)+[0]*(h+1-m)])
    #L = diag(*l)

    full_subs_list = [(x, q_cv) for x in q_cs] + [(x, p_cv) for x in p_cs] + [(x, q_rv) for x in q_rs] + [(x, p_rv) for x in p_rs] + [(k, v) for k, v in zip(d_0, d_0v)]

    i, j, r = symbols("i, j, k", integer=True)

    #C_0f = Piecewise((0, i>j),(p_c-p_c*q_c*(Rational(10, 9) - 10**(1+i-h)/9), Eq(i,j)),(p_c*q_c/(10**(j-i-1)), i<j))
    #R_0f = Piecewise((0, i>j),(p_r/4-p_r/4*q_r*(Rational(10, 9) - 10**(1+i-h)/9), Eq(i,j)),(p_r*q_r/(4*10**(j-i-1)), i<j))
    Cf = Piecewise((0, i>j),(p_c-p_c*q_c*(Rational(10, 9) - 10**(2+i-h)/9), Eq(i,j)),(p_c*q_c/(10**(j-i-1)), i<j))
    Rf = Piecewise((0, i>j),(p_r/4-p_r/4*q_r*(Rational(10, 9) - 10**(2+i-h)/9), Eq(i,j)),(p_r*q_r/(4*10**(j-i-1)), i<j))

    #Cf = piecewise_fold(Piecewise((C_0f, i<m-1),(0, (i>=m-1) & Ne(i,j)),(1, (i>=m-1) & Eq(i,j))))
    #Rf = piecewise_fold(Piecewise((R_0f, i<m-1),(0, (i>=m-1) & Ne(i,j)),(1, (i>=m-1) & Eq(i,j))))

    def apply_column_based_effects(L):
        return Matrix([[L[:,iv].subs([(q_c, q_cs[iv]), (p_c, p_cs[iv]), (q_r, q_rs[iv]), (p_r, p_rs[iv])]) for iv in range(L.shape[1])]])

    C = FunctionMatrix(h, h, Lambda((j, i), Cf)).as_explicit()
    R = FunctionMatrix(h, h, Lambda((j, i), Rf)).as_explicit()
    C = apply_column_based_effects(C)
    R = apply_column_based_effects(R)

    if debug:
        display(C.subs(full_subs_list))
        display(R.subs(full_subs_list))

    #CR = simplify(C * R)
    CR = (C * R)
    T = CR * diag(*l) + eye(h) - diag(*l)

    if debug:
        display(T.subs(full_subs_list))

    def L_eigenvectors(L):
        eigenvectors = []
        for r in range(1, L.shape[0]+1):
            L_minus = L - L[r-1,r-1] * eye(L.shape[0])
            eigenvector = [0] * (r - 1) + [1]
            for jv in range(r+1, L.shape[0]+1):
                s = 0
                for iv in range(1, jv):
                    s += L_minus[jv-1,iv-1] * eigenvector[iv-1]
                if L_minus[jv-1,jv-1]==0:
                    assert s==0
                    eigenvector.append(0)
                else:   
                    eigenvector.append(-s/L_minus[jv-1,jv-1])
            eigenvectors.append(eigenvector)
        return Matrix(eigenvectors).T

    def L_inverse(L):
        columns = []
        for r in range(1, L.shape[0]+1):
            column = [0] * (r - 1) + [1 / L[r-1,r-1]]
            for jv in range(r+1, L.shape[0]+1):
                s = 0
                for iv in range(1, jv):
                    s += L[jv-1,iv-1] * column[iv-1]
                column.append(-s/L[jv-1,jv-1])
            columns.append(column)
        return Matrix(columns).T

    def D_inf_val(D):
        return diag(*[1 if D[iv,iv]==1 else 0 for iv in range(D.shape[0])])

    def D_inf_sum(D):
        #return simplify(diag(*[0 if D[iv,iv]==1 else 1/(1-D[iv,iv]) for iv in range(D.shape[0])]))
        return diag(*[0 if D[iv,iv]==1 else 1/(1-D[iv,iv]) for iv in range(D.shape[0])])

    D = diag(*Matrix.diagonal(T))
    #E = simplify(L_eigenvectors(CR))
    #E_inv = simplify(L_inverse(E))
    E = L_eigenvectors(T)
    E_inv = L_inverse(E)

    #display(E)
    #display(D)
    #display(E_inv)

    d_inf = E * D_inf_val(D) * E_inv * C * d_0
    #display(d_inf)

    L = (E * D_inf_sum(D) * E_inv * C * d_0)


    #display(D_inf_sum(D))

    #subs_list = [(q_c, q_cv), (p_c, p_cv), (q_r, q_rv), (p_r, p_rv)] + [(k, v) for k, v in zip(d_0, d_0v)]

    #display(d_inf.subs(subs_list))
    #print(float(1/d_inf.subs(subs_list)[m-1]))
    #display(Lc.subs(subs_list))
    #print(float(Lc.subs(subs_list)))
    #display(Lr.subs(subs_list))
    #print(float(Lr.subs(subs_list)))

    if debug:
        print("========================================")
        display(d_inf.subs(full_subs_list))
        print(float(1/d_inf.subs(full_subs_list)[m-1]))
        display(L.subs(full_subs_list))
        print(float((l * L)[0,0].subs(full_subs_list)))
    result = float(1/d_inf.subs(full_subs_list)[m-1])

    #derivatives = [q_c, p_c, q_r, p_r]
    full_derivatives = [*q_cs] + [*p_cs] + [*q_rs] + [*p_rs]
    full_derivatives_bundle = list(zip([*q_cs], list(range(q_cs.shape[1])))) + list(zip([*p_cs], list(range(p_cs.shape[1])))) + \
                                         list(zip([*q_rs], list(range(q_rs.shape[1])))) + list(zip([*p_rs], list(range(p_rs.shape[1]))))
    if gradients:
        for x, g in zip(full_derivatives, [d_inf.diff(dt)[m-1,0] / (L+d_0)[i,0] for dt, i in full_derivatives_bundle]):
            print("========================================")
            print(x)
            print(float(g.subs(full_subs_list)))
    
    return result

In [5]:
quality_evaluation(0, 5, False, False, 0, 4, debug=True)

Matrix([
[0.8889,     0,    0,   0,   0],
[   0.1, 0.889,    0,   0,   0],
[  0.01,   0.1, 0.89,   0,   0],
[ 0.001,  0.01,  0.1, 0.9,   0],
[0.0001, 0.001, 0.01, 0.1, 1.0]])

Matrix([
[8889/40000,        0,      0,    0,   0],
[      1/40, 889/4000,      0,    0,   0],
[     1/400,     1/40, 89/400,    0,   0],
[    1/4000,    1/400,   1/40, 9/40,   0],
[   1/40000,   1/4000,  1/400, 1/40, 1/4]])

Matrix([
[0.1975358025,          0,        0,      0, 0],
[   0.0444475, 0.19758025,        0,      0, 0],
[  0.00694725,   0.044475, 0.198025,      0, 0],
[ 0.000947225,  0.0069725,  0.04475, 0.2025, 0],
[0.0001222225, 0.00097225, 0.007225, 0.0475, 1]])

Matrix([
[                  0],
[                  0],
[                  0],
[                  0],
[0.00093573888916793]])

1068.6741906058978


Matrix([
[    1.10771297058396],
[   0.185981304996631],
[  0.0323789114263491],
[ 0.00601249481306718],
[-0.00171904198912566]])

1.3320856818200086


1068.6741906058978

In [6]:
quality_evaluation(0, 5, False, False, 4, 0, debug=True)

Matrix([
[1.4,   0,   0,   0,   0],
[  0, 1.4,   0,   0,   0],
[  0,   0, 1.4,   0,   0],
[  0,   0,   0, 1.4,   0],
[  0,   0,   0,   0, 1.4]])

Matrix([
[8889/40000,        0,      0,    0,   0],
[      1/40, 889/4000,      0,    0,   0],
[     1/400,     1/40, 89/400,    0,   0],
[    1/4000,    1/400,   1/40, 9/40,   0],
[   1/40000,   1/4000,  1/400, 1/40, 1/4]])

Matrix([
[0.311115,       0,      0,     0, 0],
[   0.035, 0.31115,      0,     0, 0],
[  0.0035,   0.035, 0.3115,     0, 0],
[ 0.00035,  0.0035,  0.035, 0.315, 0],
[  3.5e-5, 0.00035, 0.0035, 0.035, 1]])

Matrix([
[                   0],
[                   0],
[                   0],
[                   0],
[0.000244472408699892]])

4090.4411476044124


Matrix([
[     2.03226953700545],
[    0.103258232990174],
[   0.0155802201112238],
[  0.00236205253204389],
[-0.000714810235739094]])

2.153470042638893


4090.4411476044124

In [1]:
output_dump: str = ""
for starting_tier in range(0, 4):
    for target_tier in range(starting_tier+1, 5+1):
        for max_qual in [False, True]:
            for max_assembler_modules in [4, 5]:
                for base_prod in [False, True]:
                    best_module_setup_quals = 0
                    best_module_setup_prods = 0
                    best_result = 10000000000000000000
                    for prods in range(max_assembler_modules+1):
                        quals = max_assembler_modules-prods

                        result = quality_evaluation(starting_tier, target_tier, max_qual, base_prod, prods, quals)

                        if result < best_result:
                            best_result = result
                            best_module_setup_quals = quals
                            best_module_setup_prods = prods
                        
                    print(str(starting_tier+1)+"->"+str(target_tier)+(" with max qual " if max_qual else " at base qual ")+" with "+str(max_assembler_modules)+" modules and "+str(50 if base_prod else 0)+" base productivty:\n"+\
                          "Proper setup is: "+str(best_module_setup_prods)+" productivity modules and "+str(best_module_setup_quals)+" quality modules, resulting in "+str(best_result)+" inputs per 1 output.")
                    output_dump += str(starting_tier+1)+", "+str(target_tier)+", "+str((5 if max_qual else 1))+", "+str(max_assembler_modules)+", "+str((50 if base_prod else 0))+", "+str(best_module_setup_prods)+", "+str(best_module_setup_quals)+", "+str(best_result)+"\n"

0->1 at base qual  with 4 modules and 0 base productivty:
Proper setup is: 4 productivity modules and 0 quality modules, resulting in 0.7142857142857143 inputs per 1 output.
0->1 at base qual  with 4 modules and 50 base productivty:
Proper setup is: 4 productivity modules and 0 quality modules, resulting in 0.5263157894736842 inputs per 1 output.
0->1 at base qual  with 5 modules and 0 base productivty:
Proper setup is: 5 productivity modules and 0 quality modules, resulting in 0.6666666666666666 inputs per 1 output.
0->1 at base qual  with 5 modules and 50 base productivty:
Proper setup is: 5 productivity modules and 0 quality modules, resulting in 0.5 inputs per 1 output.
0->1 with max qual  with 4 modules and 0 base productivty:
Proper setup is: 4 productivity modules and 0 quality modules, resulting in 0.5 inputs per 1 output.
0->1 with max qual  with 4 modules and 50 base productivty:
Proper setup is: 4 productivity modules and 0 quality modules, resulting in 0.4 inputs per 1 outp

In [2]:
print(output_dump)

0, 1, 1, 4, 0, 4, 0, 0.7142857142857143
0, 1, 1, 4, 50, 4, 0, 0.5263157894736842
0, 1, 1, 5, 0, 5, 0, 0.6666666666666666
0, 1, 1, 5, 50, 5, 0, 0.5
0, 1, 5, 4, 0, 4, 0, 0.5
0, 1, 5, 4, 50, 4, 0, 0.4
0, 1, 5, 5, 0, 5, 0, 0.4444444444444444
0, 1, 5, 5, 50, 5, 0, 0.36363636363636365
0, 2, 1, 4, 0, 0, 4, 6.70083771097438
0, 2, 1, 4, 50, 0, 4, 3.618895785554276
0, 2, 1, 5, 0, 0, 5, 5.633484138539265
0, 2, 1, 5, 50, 0, 5, 3.110385723055909
0, 2, 5, 4, 0, 4, 0, 2.5555
0, 2, 5, 4, 50, 4, 0, 1.4044
0, 2, 5, 5, 0, 5, 0, 1.8764938271604938
0, 2, 5, 5, 50, 5, 0, 1.0651570247933886
0, 3, 1, 4, 0, 0, 4, 38.510302169265245
0, 3, 1, 4, 50, 0, 4, 16.408395871868116
0, 3, 1, 5, 0, 0, 5, 31.333559331677943
0, 3, 1, 5, 50, 0, 5, 13.50770431618352
0, 3, 5, 4, 0, 3, 1, 8.203013130499016
0, 3, 5, 4, 50, 4, 0, 3.6487372316808724
0, 3, 5, 5, 0, 4, 1, 5.523445052626671
0, 3, 5, 5, 50, 5, 0, 2.4125383424922133
0, 4, 1, 4, 0, 0, 4, 208.55212391623706
0, 4, 1, 4, 50, 0, 4, 70.82649728239322
0, 4, 1, 5, 0, 0, 5, 161

In [4]:
[*q_cs]

[q_{c_1}, q_{c_2}, q_{c_3}, q_{c_4}, q_{c_5}]

In [3]:
list(range(1, 5))

[1, 2, 3, 4]